In [15]:
import asyncio
from decimal import Decimal
from exchange import OkxExchange
import nest_asyncio
nest_asyncio.apply()

async def close(self):
    if not self._session.closed:
        await self._session.close()

async def __aenter__(self):
    return self

async def __aexit__(self, exc_type, exc, tb):
    await self.close()

async def main():
    async with OkxExchange() as exchange:
        # Тест получения баланса
        print("\n=== Testing get_balance ===")
        balance = await exchange.get_balance("USDT")
        print(f"USDT balance: {balance} ({type(balance)})")
        
        # Тест получения цены
        print("\n=== Testing get_symbol_price ===")
        symbol = "BTC-USDT"
        price = await exchange.get_symbol_price(symbol)
        print(f"{symbol} price: {price} ({type(price)})")
        
        # Тест ордеров
        print("\n=== Testing orders ===")
        test_price = Decimal(price * 0.5)  # 50% от текущей цены
        
        # Размещение ордера
        order_id = await exchange.place_order(
            symbol=symbol,
            side="buy",
            price=test_price,
            size=Decimal("0.001"),
            td_mode="isolated"
        )
        print(f"Placed order ID: {order_id}")
        
        # Отмена одного ордера
        if order_id:
            canceled = await exchange.cancel_order(symbol, order_id)
            print(f"Cancel single order: {'Success' if canceled else 'Failed'}")
        
        # Массовое создание и отмена ордеров
        order_ids = []
        for _ in range(3):
            oid = await exchange.place_order(
                symbol=symbol,
                side="buy",
                price=test_price,
                size=Decimal("0.001")
            )
            if oid:
                order_ids.append(oid)
                print(f"Created order: {oid}")
        
        # Отмена всех ордеров
        canceled_ids = await exchange.cancel_all_orders(symbol)
        print(f"Canceled orders: {canceled_ids}")
        print(f"All created orders canceled: {set(order_ids) == set(canceled_ids)}")

await main()

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001758550E350>


TypeError: 'OkxExchange' object does not support the asynchronous context manager protocol